In [4]:
#Librerías de análisis de datos:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score, roc_auc_score

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import sys
!{sys.executable} -m pip install xgboost lightgbm catboost

In [5]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [6]:
# Buenas prácticas.
path_to_data = "./stroke_dataset.csv"
data_stroke = pd.read_csv(path_to_data)

In [7]:
#Cantidad de entradas y columnas del dataset.
data_stroke.shape

(4981, 11)

In [8]:
data_stroke.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [9]:
data_stroke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   object 
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   object 
 5   work_type          4981 non-null   object 
 6   Residence_type     4981 non-null   object 
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   object 
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 428.2+ KB


In [10]:
data_stroke.nunique()

gender                  2
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               4
Residence_type          2
avg_glucose_level    3895
bmi                   342
smoking_status          4
stroke                  2
dtype: int64

In [11]:
data_stroke.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [12]:
categoricas=["gender", "ever_married","work_type","Residence_type","smoking_status"]
numericas=["age", "hypertension","heart_disease","avg_glucose_level","bmi","stroke"]

In [13]:
data_stroke.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,4981.000000,4981.000000,4981.000000,4981.000000,4981.000000,4981.000000
mean,43.419859,0.096165,0.055210,105.943562,28.498173,0.049789
std,22.662755,0.294848,0.228412,45.075373,6.790464,0.217531
min,0.080000,0.000000,0.000000,55.120000,14.000000,0.000000
25%,25.000000,0.000000,0.000000,77.230000,23.700000,0.000000
50%,45.000000,0.000000,0.000000,91.850000,28.100000,0.000000
75%,61.000000,0.000000,0.000000,113.860000,32.600000,0.000000
max,82.000000,1.000000,1.000000,271.740000,48.900000,1.000000


In [14]:
data_stroke.duplicated().sum()

0

## Preprocesamiento: 

In [16]:
## Preprocesamiento: 

def onehot_encode(df,column):
    df = df.copy()
    
    dummies = pd.get_dummies( df [column],prefix=column)
    df = pd.concat([df,dummies], axis=1)
    df = df.drop(column,axis=1)
    
    return df 
    

In [17]:
#Hacemos función que hace una copia como primer paso

def preprocess_inputs(df):
    df = df.copy()

#2.- Si quicieramos eliminar la columna 
    # df = df.drop("id", axis=1)
    
#3.- Despues de identificar las clases  dentro de cada columna, trabajamos los binarios
    df["ever_married"] = df["ever_married"].replace({"No":0,"Yes":1})
    df["gender"] = df["gender"].replace({"Male":0,"Female":1})
    df["Residence_type"] = df["Residence_type"].replace({"Urban":0,"Rural":1})
    
#4.-Haciendo Onehot_encode
    for column in ["work_type","smoking_status"]:
        df = onehot_encode(df,column = column)
        
        
#5.- Separamos la columna stroke (variable a predecir) 
    y = df['stroke']
    X = df.drop('stroke', axis=1)
    
#6.- Definiendo Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
#7.- Escalamos las variables Xtest, Xtrain  
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
   
    return X_train, X_test, y_train, y_test

In [18]:
X_train, X_test, y_train, y_test = preprocess_inputs(data_stroke)

In [19]:
X_train.var()

gender                            1.000287
age                               1.000287
hypertension                      1.000287
heart_disease                     1.000287
ever_married                      1.000287
Residence_type                    1.000287
avg_glucose_level                 1.000287
bmi                               1.000287
work_type_Govt_job                1.000287
work_type_Private                 1.000287
work_type_Self-employed           1.000287
work_type_children                1.000287
smoking_status_Unknown            1.000287
smoking_status_formerly smoked    1.000287
smoking_status_never smoked       1.000287
smoking_status_smokes             1.000287
dtype: float64

In [20]:
y_train 

2117    0
332     0
3915    0
4610    0
4485    0
       ..
2895    0
2763    0
905     0
3980    0
235     0
Name: stroke, Length: 3486, dtype: int64

## ENTRENAMOS


In [22]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")



                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                              LightGBM trained.
                              CatBoost trained.


In [23]:
y_train.value_counts()

0    3315
1     171
Name: stroke, dtype: int64

In [73]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_pred = model.predict(X_test)
    
    y_pred_train = model.predict(X_train)
    
    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),roc_auc_score(y_test, y_pred) ))
    
    print( confusion_matrix(y_test, y_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t              Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_train, y_pred_train) * 100, f1_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train) ))
    #Hemos quitado la matriz de confusion
    #,recall_score(y_train, y_pred),precision_score(y_train, y_pred)
    print(confusion_matrix(y_train, y_pred_train))
    
    print(
    np.abs(((((accuracy_score(y_train, y_pred_train))-accuracy_score(y_test, y_pred))/(accuracy_score(y_test, y_pred)) *100)))
       )
           

Model Performance
-----------------

                   Logistic Regression Accuracy: 74.247%
				       F1-Score: 0.24951
				              Recall: 0.83117
				       Precision: 0.14679
				       Roc_Au: 0.78441
				
[[1046  372]
 [  13   64]]

                   Logistic Regression Accuracy_train: 72.719%
				       F1-Score_train: 0.22871
				              Recall_Train: 0.82456
				       Precision_Train: 0.13277
				       Roc_Au_Train: 0.77337
				
[[2394  921]
 [  30  141]]
2.0580391062318726

                   K-Nearest Neighbors Accuracy: 85.619%
				       F1-Score: 0.19476
				              Recall: 0.33766
				       Precision: 0.13684
				       Roc_Au: 0.61100
				
[[1254  164]
 [  51   26]]

                   K-Nearest Neighbors Accuracy_train: 92.025%
				       F1-Score_train: 0.55161
				              Recall_Train: 1.00000
				       Precision_Train: 0.38085
				       Roc_Au_Train: 0.95807
				
[[3037  278]
 [   0  171]]
7.4826090074584055

                         

In [71]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_pred = model.predict(X_test)
    
    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred)
           ))
    #Hemos quitado la matriz de confusion
    #,recall_score(y_train, y_pred),precision_score(y_train, y_pred)
           

Model Performance
-----------------

                   Logistic Regression Accuracy: 74.247%
				       F1-Score: 0.24951

                   K-Nearest Neighbors Accuracy: 85.619%
				       F1-Score: 0.19476

                         Decision Tree Accuracy: 91.906%
				       F1-Score: 0.11679

Support Vector Machine (Linear Kernel) Accuracy: 72.843%
				       F1-Score: 0.23970

   Support Vector Machine (RBF Kernel) Accuracy: 78.528%
				       F1-Score: 0.23022

                        Neural Network Accuracy: 87.291%
				       F1-Score: 0.22764

                         Random Forest Accuracy: 94.314%
				       F1-Score: 0.10526

                     Gradient Boosting Accuracy: 79.465%
				       F1-Score: 0.22670

                               XGBoost Accuracy: 92.375%
				       F1-Score: 0.18571

                              LightGBM Accuracy: 90.903%
				       F1-Score: 0.20000

                              CatBoost Accuracy: 91.237%
				       F1-Score: 0.14379


## OVERSAMPLING


In [26]:
y_train.value_counts()

0    3315
1     171
Name: stroke, dtype: int64

In [79]:
oversampled_data = pd.concat([X_train, y_train], axis=1).copy()

num_samples = y_train.value_counts()[0] - y_train.value_counts()[1]
new_samples = oversampled_data.query("stroke == 1").sample(num_samples, replace=True, random_state=1)

oversampled_data = pd.concat([oversampled_data, new_samples], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

##Cambios segur Rb
y_oversampled = oversampled_data['stroke']
X_oversampled = oversampled_data.drop('stroke', axis=1)

#Def Train-Test-Split

X_oversampled_train, X_oversampled_test, y_oversampled_train, y_oversampled_test = train_test_split(X_oversampled, y_oversampled, train_size=0.7, shuffle=True, random_state=1)

In [80]:
X_oversampled_train.var()

gender                            1.011813
age                               0.938879
hypertension                      1.711155
heart_disease                     1.998487
ever_married                      0.795716
Residence_type                    0.998616
avg_glucose_level                 1.413906
bmi                               0.870330
work_type_Govt_job                1.034610
work_type_Private                 0.999041
work_type_Self-employed           1.244830
work_type_children                0.592118
smoking_status_Unknown            0.891125
smoking_status_formerly smoked    1.212737
smoking_status_never smoked       0.977086
smoking_status_smokes             1.097818
dtype: float64

In [81]:
y_oversampled_train

3253    1
4972    0
2408    0
6594    1
2761    0
       ..
905     1
5192    0
3980    0
235     0
5157    0
Name: stroke, Length: 4641, dtype: int64

In [28]:
oversampled_data 

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_Govt_job,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,0.860964,0.760070,-0.329816,4.153470,0.709390,1.009801,0.301358,1.210494,2.5208,-1.147292,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
1,0.860964,-0.296505,-0.329816,-0.240763,0.709390,-0.990294,-0.192419,-1.199677,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,1.309562,-0.420131,0
2,0.860964,1.200310,-0.329816,-0.240763,0.709390,-0.990294,1.957183,0.813698,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,1
3,-1.161489,0.451902,-0.329816,-0.240763,-1.409662,1.009801,-0.005382,-0.170945,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,0
4,0.860964,1.508478,-0.329816,-0.240763,0.709390,1.009801,-1.014280,-0.215034,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6625,0.860964,0.319830,-0.329816,-0.240763,0.709390,1.009801,-0.209362,-1.038019,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,-0.465568,1.309562,-0.420131,0
6626,-1.161489,1.068238,-0.329816,-0.240763,0.709390,-0.990294,-0.636906,-0.156249,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
6627,-1.161489,1.332382,-0.329816,-0.240763,0.709390,-0.990294,-0.743847,0.402205,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
6628,-1.161489,-0.428577,-0.329816,-0.240763,-1.409662,-0.990294,0.037526,0.269940,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,0


In [82]:
y_train_oversampled.value_counts()

1    3315
0    3315
Name: stroke, dtype: int64

In [84]:
## Entrenando los modelos con la data balanceada
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}


for name, model in models.items():
    model.fit(X_oversampled_train, y_oversampled_train)
    print(name + " Entrenado.")


                   Logistic Regression Entrenado.
                   K-Nearest Neighbors Entrenado.
                         Decision Tree Entrenado.
Support Vector Machine (Linear Kernel) Entrenado.
   Support Vector Machine (RBF Kernel) Entrenado.
                        Neural Network Entrenado.
                         Random Forest Entrenado.
                     Gradient Boosting Entrenado.
                               XGBoost Entrenado.
                              LightGBM Entrenado.
                              CatBoost Entrenado.


In [85]:
y_oversampled_train.value_counts()

1    2322
0    2319
Name: stroke, dtype: int64

In [48]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_oversampled_pred = model.predict(X_oversampled_test)
    
    y_oversampled_pred_train = model.predict(X_oversampled_train)


    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),roc_auc_score(y_test, y_pred) ))
    
    print( confusion_matrix(y_test, y_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t        Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_train, y_pred_train) * 100, f1_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train) ))
   # print(
   # np.abs((((accuracy_score(y_test, y_pred))/(accuracy_score(y_train, y_pred_train)) *100)))
    #      ) Confusion_Matrix: {:.5f}\n\t\t\t\t Confusion_Matrix_Train: {:.5f}\n\t\t\t\t,confusion_matrix(y_test,y_pred),
    
    
    print( confusion_matrix(y_train, y_pred_train))
    
    print(
    "\n" + name + " Overfitting:\n\t\t\t\t",
    np.abs(((((accuracy_score(y_train, y_pred_train))-accuracy_score(y_test, y_pred))/(accuracy_score(y_test, y_pred)) *100)))
       )

Model Performance
-----------------

                   Logistic Regression Accuracy: 74.247%
				       F1-Score: 0.24951
			              Recall: 0.83117
				       Precision: 0.14679
				       Roc_Au: 0.78441
				
[[1046  372]
 [  13   64]]

                   Logistic Regression Accuracy_train: 72.719%
				       F1-Score_train: 0.22871
				        Recall_Train: 0.82456
				       Precision_Train: 0.13277
				       Roc_Au_Train: 0.77337
				
[[2394  921]
 [  30  141]]

                   Logistic Regression Overfitting:
				 2.0580391062318726

                   K-Nearest Neighbors Accuracy: 85.619%
				       F1-Score: 0.19476
			              Recall: 0.33766
				       Precision: 0.13684
				       Roc_Au: 0.61100
				
[[1254  164]
 [  51   26]]

                   K-Nearest Neighbors Accuracy_train: 92.025%
				       F1-Score_train: 0.55161
				        Recall_Train: 1.00000
				       Precision_Train: 0.38085
				       Roc_Au_Train: 0.95807
				
[[3037  278]
 [   0  171]]



In [86]:
######Test con valores oversampled#####
print("Model Performance\n-----------------")
for name, model in models.items():
    y_oversampled_pred = model.predict(X_oversampled_test)
    
    y_oversampled_pred_train = model.predict(X_oversampled_train)


    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_oversampled_test, y_oversampled_pred) * 100, f1_score(y_oversampled_test, y_oversampled_pred),recall_score(y_oversampled_test, y_oversampled_pred),precision_score(y_oversampled_test, y_oversampled_pred),roc_auc_score(y_oversampled_test, y_oversampled_pred) ))
    
    print( confusion_matrix(y_oversampled_test, y_oversampled_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t        Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_oversampled_train, y_oversampled_pred_train) * 100, f1_score(y_oversampled_train, y_oversampled_pred_train),recall_score(y_oversampled_train, y_oversampled_pred_train),precision_score(y_oversampled_train, y_oversampled_pred_train),roc_auc_score(y_oversampled_train, y_oversampled_pred_train) ))
   # print(
   # np.abs((((accuracy_score(y_test, y_pred))/(accuracy_score(y_train, y_pred_train)) *100)))
    #      ) Confusion_Matrix: {:.5f}\n\t\t\t\t Confusion_Matrix_Train: {:.5f}\n\t\t\t\t,confusion_matrix(y_test,y_pred),
    
    
    print( confusion_matrix(y_oversampled_train, y_oversampled_pred_train))
    
    print(
    "\n" + name + " Overfitting:\n\t\t\t\t",
    np.abs(((((accuracy_score(y_oversampled_train, y_oversampled_pred_train))-accuracy_score(y_oversampled_test, y_oversampled_pred))/(accuracy_score(y_oversampled_test, y_oversampled_pred)) *100)))
       )

Model Performance
-----------------

                   Logistic Regression Accuracy: 78.482%
				       F1-Score: 0.79383
			              Recall: 0.82981
				       Precision: 0.76085
				       Roc_Au: 0.78488
				
[[737 259]
 [169 824]]

                   Logistic Regression Accuracy_train: 77.591%
				       F1-Score_train: 0.78819
				        Recall_Train: 0.83333
				       Precision_Train: 0.74768
				       Roc_Au_Train: 0.77587
				
[[1666  653]
 [ 387 1935]]

                   Logistic Regression Overfitting:
				 1.1348036972636548

                   K-Nearest Neighbors Accuracy: 90.447%
				       F1-Score: 0.91268
			              Recall: 1.00000
				       Precision: 0.83939
				       Roc_Au: 0.90462
				
[[806 190]
 [  0 993]]

                   K-Nearest Neighbors Accuracy_train: 94.441%
				       F1-Score_train: 0.94737
				        Recall_Train: 1.00000
				       Precision_Train: 0.90000
				       Roc_Au_Train: 0.94437
				
[[2061  258]
 [   0 2322]]

        

In [87]:
#Para usar GridSearch

from sklearn.model_selection import GridSearchCV


#Creando la Gridsearch


